# Notebook for Airconics examples 

IMPORTANT!: ipython notebook bug requires first code cells to be run manually (i.e. not with 'Run All' Cells). After this, all cells can be rerun with 'Run all below'. 

In [ ]:
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()

## Airfoils

In [ ]:
LEPoint = [0., 0., 0.] 
ChordLength = 1

AirfoilSeligName = 'dae11'

# Instantiate class to set up a generic airfoil with these basic parameters
Af1 = airconics.primitives.Airfoil(LEPoint, ChordLength=3., SeligProfile=AirfoilSeligName)
Af1_disp = display.DisplayShape(Af1.Curve, update=True);

In [ ]:
# Remove Airfoil:
display.Context.Erase(Af1_disp.GetObject().GetHandle())
Af1.Curve.GetObject().Delete()   # Not sure if this truly collects the garbage yet

# Or Clear the display:
display.EraseAll()

## Simple Lofted Wing

In [ ]:
# Add NACA 4 digit airfoils to loft between:
Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')
display.DisplayShape(Af2.Curve, update=True, color='GREEN');

Af3 = airconics.primitives.Airfoil([0., 5., 0.], ChordLength=1., Naca4Profile='0012')
display.DisplayShape(Af3.Curve, update=True, color='GREEN');

Af4 = airconics.primitives.Airfoil([0., 6., 0.2], ChordLength=0.2, Naca4Profile='0012')
display.DisplayShape(Af4.Curve, update=True, color='GREEN');

surf = act.AddSurfaceLoft([Af2, Af3, Af4])

# # Note that surf is a TOPO_DS Shape, and hence no surf.Shape is required for display
display.DisplayShape(surf, update=True);

In [ ]:
display.EraseAll()

## Lifting Surface, Transonic Airliner

TODO: I tried making the lifting surface loft through airfoil sections as a solid loft, however the output stp file was not correct in Rhino (may be because TE and ends are not closed)

In [ ]:
# Import all example functional definitions for the Common Research Model (CRM) Wing:
from wing_example_transonic_airliner import *

# Position of the apex of the wing
P = (0,0,0)

# Class definition
NSeg = 10
ChordFactor = 1
ScaleFactor = 50

# Generate (surface building is done during construction of the class)
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
    myDihedralFunctionAirliner, 
    myTwistFunctionAirliner, 
    myChordFunctionAirliner, 
    myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

# Here the TOPO_DS Shape to be displayed is stored in the Lifting surface 'Shape' Attribute
display.DisplayShape(Wing.Shape, update=True)

### File input/output

In [ ]:
# Write Output STEP file e.g. for viewing in Rhino:
act.export_STEPFile([Wing.Shape], 'wing.stp')

In [ ]:
# OR use python 'pickle' (Though for files which will be sent, STEP is probably better)
import pickle

pickle.dump(Wing, open("wing.p", "wb"))

del(Wing)
display.EraseAll()

LoadedWing = pickle.load(open("wing.p", "rb"))
display.DisplayShape(LoadedWing.Shape, update=True, color='Blue')

## Random things I tried out (Needs removing)

In [ ]:
# from OCC.gp import gp_Lin
# from OCC.ShapeConstruct import shapeconstruct

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.curve.GetHandle()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = gp_Lin(EP, SP)

# shapeconstruct.JoinCurves(h.GetObject(), closure.)

# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# from OCC.gp import gp_Lin, gp_Ax1, gp_Dir
# from OCC.ShapeConstruct import shapeconstruct
# from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.Curve.GetObject()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = BRepBuilderAPI_MakeEdge(gp_Lin(gp_Ax1(EP, gp_Dir(SP.XYZ()))) ).Edge()

# display.DisplayShape(closure, update=True)
# display.DisplayShape(Af2.Curve, update=True)